Google drive

In [11]:
from google.colab import drive
drive.mount('/content/drive')

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


## spec

In [1]:
!nvidia-smi

Mon Sep  6 07:02:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get GitHub repository files and install python packages

In [2]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b WJ
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 257, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 257 (delta 142), reused 154 (delta 40), pack-reused 0
Receiving objects: 100% (257/257), 2.95 MiB | 13.25 MiB/s, done.
Resolving deltas: 100% (142/142), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 5.5 MB/s 
     |████████████████████████████████| 282 kB 35.7 MB/s 
     |████████████████████████████████| 2.8 MB 33.5 MB/s 
     |████████████████████████████████| 919 kB 20.3 MB/s 
     |████████████████████████████████| 119 kB 38.4 MB/s 
     |████████████████████████████████| 243 kB 16.0 MB/s 
     |████████████████████████████████| 50 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 36.4 MB/s 
     |████████████████████████████████| 3.3 MB 34.8 MB/s 
     |████████████████████████████████| 636 kB 36.9 MB/s 
     |██████████████████████████

## model and tokenizer



In [3]:
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [4]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer

# model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", **model_config)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458 [00:00<?, ?B/s]

## datasets

In [5]:
from data_loader import get_data_loaders

train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)

Using custom data configuration default-cfa12129877ac758


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cfa12129877ac758/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

## metric

In [6]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

## Do Train

In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
from transformers import set_seed

set_seed(42)

In [12]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=f'{path_to_drive}/koelectra-small-v3-discriminator-test',
    seed=42,
    num_train_epochs=200,
    learning_rate=1e-4,
    weight_decay=0.0,
    gradient_accumulation_steps=1,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,

    # checkpoint
    overwrite_output_dir=True,
    save_strategy='steps',
    save_steps=1000,

    # evaluation
    evaluation_strategy='steps',
    eval_steps=1000,
    metric_for_best_model="f1",

    # early stopping
    load_best_model_at_end=True
)

In [11]:
from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
scheduler = AdafactorSchedule(optimizer)

In [13]:
from trainer import ImbalancedSamplerTrainer

def model_init():
    return ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", **model_config, return_dict=True)

trainer = ImbalancedSamplerTrainer(
    model_init=model_init,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
    # optimizers=(optimizer, scheduler)
)

loading configuration file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/bd0f09888c5a5619ddb9de81d4a9936a94e5f45064f9a23ba6d39241ceebce02.d2485d28e5c07ca60bfa4fe84af673e0df83401e5c56bcdd991878cb4966eb34
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary

Downloading:   0%|          | 0.00/56.6M [00:00<?, ?B/s]

storing https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/052bc3ecf8c8484f1519650794b32b6d2c70750bcba71d1f763951514b5cf0c8.84bb33167d2e89d46f4cde129b2f4c447618ac33ac46f2012ee9e5e706fec112
creating metadata file for /root/.cache/huggingface/transformers/052bc3ecf8c8484f1519650794b32b6d2c70750bcba71d1f763951514b5cf0c8.84bb33167d2e89d46f4cde129b2f4c447618ac33ac46f2012ee9e5e706fec112
loading weights file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/052bc3ecf8c8484f1519650794b32b6d2c70750bcba71d1f763951514b5cf0c8.84bb33167d2e89d46f4cde129b2f4c447618ac33ac46f2012ee9e5e706fec112
Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.

# install ray

In [14]:
!pip install "ray[tune]" transformers datasets scipy sklearn torch

     |████████████████████████████████| 49.6 MB 6.2 kB/s 
     |████████████████████████████████| 72 kB 476 kB/s 
     |████████████████████████████████| 124 kB 55.3 MB/s 


# haperparameter search

In [ ]:
trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    n_trials=10 # number of trials
)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
/usr/local/lib/python3.7/dist-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


== Status ==
Memory usage on this node: 2.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-09-06_07-07-38
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------|
| _objective_1e810_00000 | RUNNING  |       |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_1e810_00001 | PENDING  |       |     1.56207e-05 |                  2 |                            16 |  7.08379 |
| _objective_1e810_00002 | PENDING  |       |  

(pid=478) Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
(pid=478) - This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=478) - This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=478) Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3

Result for _objective_1e810_00000:
  {}
  


(pid=477) Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
(pid=477) - This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=477) - This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=477) Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3

...

 60%|█████▉    | 3000/5032 [35:09<22:37,  1.50it/s]


(pid=477) {'loss': 0.4188, 'learning_rate': 6.307879481101004e-06, 'epoch': 1.19}


(pid=477) 
  0%|          | 0/140 [00:00<?, ?it/s]
(pid=477) 
  1%|▏         | 2/140 [00:00<00:31,  4.44it/s]
(pid=477) 
  2%|▏         | 3/140 [00:00<00:43,  3.16it/s]
(pid=477) 
  3%|▎         | 4/140 [00:01<00:48,  2.78it/s]
(pid=477) 
  4%|▎         | 5/140 [00:01<00:52,  2.59it/s]
(pid=477) 
  4%|▍         | 6/140 [00:02<00:54,  2.48it/s]
(pid=477) 
  5%|▌         | 7/140 [00:02<00:55,  2.42it/s]
(pid=477) 
  6%|▌         | 8/140 [00:03<00:55,  2.38it/s]
(pid=477) 
  6%|▋         | 9/140 [00:03<00:55,  2.35it/s]
(pid=477) 
  7%|▋         | 10/140 [00:03<00:55,  2.33it/s]
(pid=477) 
  8%|▊         | 11/140 [00:04<00:55,  2.32it/s]
(pid=477) 
  9%|▊         | 12/140 [00:04<00:55,  2.31it/s]
(pid=477) 
  9%|▉         | 13/140 [00:05<00:55,  2.31it/s]
(pid=477) 
 10%|█         | 14/140 [00:05<00:54,  2.30it/s]
(pid=477) 
 11%|█         | 15/140 [00:06<00:54,  2.29it/s]
(pid=477) 
 11%|█▏        | 16/140 [00:06<00:54,  2.29it/s]
(pid=477) 
 12%|█▏        | 17/140 [00:07<00:53,  2.29it/

Result for _objective_1e810_00001:
  date: 2021-09-06_07-44-03
  done: false
  epoch: 1.19
  eval_accuracy: 0.8423524150268337
  eval_f1: 0.6287487526967918
  eval_loss: 0.41695380210876465
  eval_runtime: 61.2026
  eval_samples_per_second: 73.069
  eval_steps_per_second: 2.287
  experiment_id: ac070bd67d2642f097c5da25ca5c43d8
  hostname: c91990d96aa3
  iterations_since_restore: 3
  node_ip: 172.28.0.2
  objective: 1.4711011677236254
  pid: 477
  time_since_restore: 2174.494803905487
  time_this_iter_s: 724.646347284317
  time_total_s: 2174.494803905487
  timestamp: 1630914243
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: 1e810_00001
  
== Status ==
Memory usage on this node: 3.3/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-09-06_07-07-38
Number of trials: 10/10 (1 ERROR, 8 PENDING, 1 RUNNING)
+-------------

(pid=477) 
                                                   
100%|██████████| 140/140 [01:00<00:00,  2.45it/s]
                                                 
 68%|██████▊   | 3441/5032 [41:04<17:38,  1.50it/s]
